In [133]:
import pandas as pd
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [54]:
news = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [102]:
news = news.dropna(subset=['content', 'source'])
news[:5]

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [119]:
news_xinhuashe = news[news['source'].str.contains('新华')]
news_xinhuashe[:5]

,id,author,source,content,feature,title,url
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
17,89600,NaN,新华网,国歌法草案首次提请最高立法机关审议\r\n点击图片进入下一页\r\n新闻：十二届全国人大常委...,"{""type"":""观点滚动"",""site"":""参考消息"",""commentNum"":""0"",...",[及时点]我们不需要“百变老太太”,http://column.cankaoxiaoxi.com/plgd/2017/0623/...
25,89592,NaN,新华网,当地时间2017年6月17日，土耳其马尼萨省，在土耳其第1步兵训练大队司令部军营，590名土...,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",土耳其近590名士兵疑食物中毒 被送往医院救治（组图）,http://world.huanqiu.com/hot/2017-06/10866176....
27,89590,NaN,新华网,英国与欧盟的“脱欧”谈判于19日正式开始。然而此时，英国首相特雷莎·梅正面临着空前的政治压力...,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",火灾与脱欧炙烤英国首相 特雷莎·梅面临空前政治压力,http://world.huanqiu.com/hot/2017-06/10866198....
39,89578,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态,http://world.huanqiu.com/hot/2017-06/10866229....


In [118]:
news_not_xinhuashe = news[~news['source'].str.contains('新华')]
news_not_xinhuashe[:5]

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...
5,89612,张怡,中国证券报?中证网,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",金融股一枝独秀 配置价值犹存,http://www.cs.com.cn/gppd/201706/t20170623_533...


In [120]:
news_xinhuashe = news_xinhuashe['content'].tolist()

In [121]:
a = ''.join(re.findall(re.compile('\w|\d'), news_xinhuashe[0]))
b = ' '.join(jieba.cut(a))
b

'这是 6 月 18 日 在 葡萄牙 中部 大 佩德罗 冈 地区 拍摄 的 被 森林 大火 烧毁 的 汽车 新华社 记者 张立 云摄'

In [122]:
def cut(content):
    def remove_extra_char(content):
        content =  [''.join(re.findall(re.compile('\w|\d'), con)) for con in content]
        return content
    
    content = remove_extra_char(content)
    
    jieba.enable_parallel(4)
    
    return [' '.join(jieba.cut(con)) for con in content]

In [124]:
tokens_xinhuashe = cut(news_xinhuashe[:5000])
tokens_xinhuashe[:5]

['这是 6 月 18 日 在 葡萄牙 中部 大 佩德罗 冈 地区 拍摄 的 被 森林 大火 烧毁 的 汽车 新华社 记者 张立 云摄',
 '国歌 法 草案 首次 提请 最高 立法机关 审议 点击 图片 进入 下 一页 新闻 十二届 全国人大常委会 第二十八 次 会议 22 日 首次 开始 审议 国歌 法 草案 草案 对 应当 奏 唱国歌 的 场合 国歌 标准 曲谱 的 审定 发布 和 使用 奏 唱 礼仪 规范 以及 不得 奏 唱 播放 国歌 的 场合 等 做出 规范 草案 规定 在 公共场合 恶意 修改 国歌 歌词 或者 故意 以 歪曲 贬损 方式 奏 唱国歌 损害 国歌 庄严 形象 的 由 公安机关 处以 十五日 以下 拘留 新闻来源 新华社 点评 国歌 代表 了 一个 国家 的 气质 也 是 一个 国家 的 象征 为 国歌 立法 不仅 有助于 推动 实现 国旗 国徽 国歌 等 国家 象征 标志物 立法 的 完整 统一 而且 有助于 纠正 现实 中 存在 的 种种 不 规范 国歌 使用 状况 有力 捍卫 国歌 应有 的 严肃性 和 神圣 性 刘洪滨 代言 假药 月 销售额 近 百万元 点击 图片 进入 下 一页 新闻 近日 一位 名叫 刘洪滨滨 有时 写作 斌 的 老 专家 被 发现 多次 在 各大 电视台 的 健康 医药 节目 中 参与 推广 的 药品 功效 也 是 五花八门 有时 是 苗医 传人 有时 是 蒙医 传人 偶尔 还会 客串 一下 北大 专家 和 医院 退休 的 老 院长 由于 刘洪滨 出现 在 电视 画面 中 的 身份 完全 不同 也 被 网友 戏称 为 虚假 医药 广告 表演艺术家 新闻来源 北京青年报',
 '当地 时间 2017 年 6 月 17 日 土耳其 马 尼萨 省 在 土耳其 第 1 步兵 训练 大队 司令部 军营 590 名 土耳其 士兵 疑是 食物中毒 在 晚饭 后 出现 恶心 和 呕吐 症状 被 紧急 送往 医院 救治 土耳其 警方 随后 逮捕 了 19 名 相关 人员 其中 包括 为 军营 提供 食物 的 餐饮公司 负责人 来源 新华网',
 '英国 与 欧盟 的 脱欧 谈判 于 19 日 正式 开始 然而 此时 英国首相 特雷莎 梅正 面临 着 空前 的 政治 压力 不久前 的 大选 失利 让 梅 饱受 诟病 而 尘埃 

In [125]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tokens_xinhuashe)
X.shape

(5000, 54473)

In [127]:
#vectorizer.vocabulary_

In [128]:
tf_idf_vector = X.toarray()

In [129]:
tf_idf_vector.shape

(5000, 54473)

In [131]:
vectorizer.vocabulary_["这是"]

48904

In [134]:
np.where(tf_idf_vector[0])

(array([  411,  4902,  5903,  8014, 16994, 18544, 23624, 26635, 28805,
        32485, 33889, 36083, 44732, 46097, 48904]),)